In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [13]:

X= pd.read_csv('data/train.csv')
X=X.drop(X.columns[0],axis=1)
X

,Open
0,0.049579
1,0.049895
2,0.043579
3,0.036526
4,0.041368
...,...
1814,0.421368
1815,0.421263
1816,0.410316
1817,0.410632


In [26]:
a=[]
b=X['Open']
print(len(X))
a.append(b[1:4])
a

1819


[1    0.049895
 2    0.043579
 3    0.036526
 Name: Open, dtype: float64]

In [36]:


# Load time series data


# Define hyperparameter grid

window_size=3
# Define sliding window function
def sliding_window(X, window_size):
    X_windows = []
    y=[]
    for i in range(len(X)-window_size):
        X_windows.append(X[i:i+window_size])
        y.append(X[i+window_size])
    return np.array(X_windows),np.array(y)

# Apply sliding window approach to training data
X_windows,y = sliding_window(X['Open'], window_size)
X_train= pd.DataFrame(X_windows.reshape(-1, window_size))
X_train.shape,y.shape


((1816, 3), (1816,))

In [61]:
param_grid = {
    'max_depth': [None, 5,10,15,20],
    'min_samples_split': [2, 5,4, 10],
    'min_samples_leaf': [1, 2, 4],
}

# Define decision tree model
model = DecisionTreeRegressor(random_state=42)

# Perform hyperparameter tuning
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=4)
grid_search.fit(X_train, y)

# Print best hyperparameters
print('Best hyperparameters:', grid_search.best_params_)

# Train final decision tree model
final_model = DecisionTreeRegressor(**grid_search.best_params_, random_state=42)
final_model.fit(X_train, y)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV 1/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-0.002 total time=   0.0s
[CV 2/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-0.000 total time=   0.0s
[CV 3/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-0.000 total time=   0.0s
[CV 4/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-0.000 total time=   0.0s
[CV 5/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-0.000 total time=   0.0s
[CV 1/5] END max_depth=None, min_samples_leaf=1, min_samples_split=5;, score=-0.002 total time=   0.0s
[CV 2/5] END max_depth=None, min_samples_leaf=1, min_samples_split=5;, score=-0.000 total time=   0.0s
[CV 3/5] END max_depth=None, min_samples_leaf=1, min_samples_split=5;, score=-0.000 total time=   0.0s
[CV 4/5] END max_depth=None, min_samples_leaf=1, min_samples_split=5;, score=-0.000 total time=   0.0s
[CV 5/5] EN

DecisionTreeRegressor(max_depth=10, min_samples_leaf=4, min_samples_split=10,
                      random_state=42)

In [62]:
train_predict=final_model.predict(X_train)
mean_squared_error(train_predict,y)

6.281278307266e-05

In [63]:
test_data=pd.read_csv('data/test.csv')
test_data

,Unnamed: 0,Open
0,2209,0.511158
1,2210,0.512632
2,2211,0.518842
3,2212,0.514526
4,2213,0.522105
...,...,...
386,2595,0.661579
387,2596,0.662105
388,2597,0.700526
389,2598,0.722105


In [52]:
test_X_windows,y_test = sliding_window(test_data['Open'], window_size)
test_X_windows = pd.DataFrame(test_X_windows.reshape(-1, window_size))
test_X_windows
print(y_test)

[0.51452632 0.52210526 0.52210526 0.53368421 0.548      0.55663158
 0.55473684 0.56284211 0.57473684 0.58526316 0.592      0.58578947
 0.60157895 0.582      0.57473684 0.57336842 0.57673684 0.53789474
 0.52547368 0.51263158 0.51063158 0.53263158 0.52410526 0.52115789
 0.51494737 0.51263158 0.51684211 0.49915789 0.52073684 0.51642105
 0.50926316 0.52115789 0.52263158 0.52505263 0.51494737 0.52842105
 0.51547368 0.53168421 0.52505263 0.52694737 0.52073684 0.52021053
 0.52357895 0.52357895 0.51115789 0.49778947 0.48147368 0.49442105
 0.48147368 0.48821053 0.514      0.51115789 0.49915789 0.50157895
 0.54463158 0.51978947 0.51589474 0.50873684 0.50536842 0.50631579
 0.52357895 0.52842105 0.52884211 0.52263158 0.53031579 0.51926316
 0.50873684 0.51684211 0.51494737 0.51736842 0.51547368 0.54515789
 0.51642105 0.50494737 0.52978947 0.51884211 0.53031579 0.49873684
 0.49778947 0.50157895 0.48673684 0.49484211 0.49105263 0.50252632
 0.514      0.50736842 0.50063158 0.51115789 0.51210526 0.5121

In [64]:
test_y_pred = final_model.predict(test_X_windows)
test_y_pred.shape,y_test.shape

((388,), (388,))

In [65]:

# Apply sliding window approach to test data

# Predict on test data


# Evaluate performance
test_y_pred = test_y_pred.flatten()
y_test = y_test.flatten()
mse = mean_squared_error(y_test, test_y_pred)
print('Mean Squared Error:', mse)

Mean Squared Error: 0.030309334807708515
